In [ ]:
GITHUB_USERNAME = "$GITHUB_USERNAME$"
GITHUB_REF = "$GITHUB_REF$"
NOTEBOOK_TYPE = "jupyter" #"$NOTEBOOK_TYPE$"
PYTHON_VERSION = "3.8" #"$PYTHON_VERSION$"
IPYTHON_VERSION = "7.3.0" #"$IPYTHON_VERSION$"

In [ ]:
from pathlib import Path

import requests


if NOTEBOOK_TYPE == 'colab':
    # utils module doesn't exist on colab VM, so get current version from GitHub
    utils_module = Path('utils.py').resolve()
    response = requests.get(f'https://raw.githubusercontent.com/{GITHUB_USERNAME}/davos/{GITHUB_REF}/tests/utils.py')
    utils_module.write_text(response.text)
    # also need to install davos locally
    from utils import install_davos
    install_davos(source='github', ref=GITHUB_REF, fork=GITHUB_USERNAME)

In [ ]:
import davos

from utils import mark, raises, run_tests

In [ ]:
IPYTHON_SHELL = get_ipython()

# tests for `davos.implementations.ipython_post7`
**Note**: this test notebook is only run for jobs where `IPython>=7.0`

In [ ]:
def test_ipython_post7_imports():
    """
    check that functions that should've been imported from the 
    ipython_post7 module came from the right place
    """
    ipy_post7_funcs = (
        '_activate_helper', 
        '_deactivate_helper',
        'generate_parser_func'
    )
    for func_name in ipy_post7_funcs:
        func_obj = getattr(davos.implementations, func_name)
        func_module = getattr(func_obj, '__module__')
        assert func_module == 'davos.implementations.ipython_post7', (
            f"davos.implementations.{func_name} is {func_module}.{func_name}. "
            f"Expected davos.implementations.ipython_post7.{func_name}"
        )

In [ ]:
def test_activate_helper_registers_parser():
    """
    check that the `davos` parser was added to `input_transformers_post` 
    when `davos` was imported above
    """
    davos_parser = davos.implementations.full_parser
    transformers_list = IPYTHON_SHELL.input_transformers_post
    assert davos_parser in transformers_list, (
        f"{davos_parser} not in {transformers_list}"
    )

In [ ]:
def test_activate_helper_registers_parser_once():
    """
    `_activate_helper` should not register multiple instances of the 
    `davos` parser if called multiple times (without deactivating)
    """
    smuggle_func = davos.core.core.smuggle
    davos_parser = davos.implementations.full_parser
    transformers_list = IPYTHON_SHELL.input_transformers_post
    
    assert transformers_list.count(davos_parser) == 1
    
    for _ in range(5):
        davos.implementations.ipython_post7._activate_helper(smuggle_func, 
                                                            davos_parser)
    
    assert transformers_list.count(davos_parser) == 1

In [ ]:
def test_activate_helper_adds_smuggle():
    """
    `_activate_helper` should inject the `smuggle_func` it is passed 
    into the IPython user namespace as `"smuggle"`
    """    
    real_smuggle_func = davos.core.core.smuggle
    davos_parser = davos.implementations.full_parser
    
    def _fake_smuggle_func():
        pass
    
    assert IPYTHON_SHELL.user_ns['smuggle'] is real_smuggle_func is smuggle
    
    try:
        davos.implementations.ipython_post7._activate_helper(_fake_smuggle_func, 
                                                            davos_parser)
        assert smuggle is IPYTHON_SHELL.user_ns['smuggle'] is _fake_smuggle_func
    finally:
        # regardless of test outcome, make sure original value of 
        # "smuggle" is restored
        IPYTHON_SHELL.user_ns['smuggle'] = real_smuggle_func

In [ ]:
def test_deactivate_helper_removes_parser():
    """
    `_deactivate_helper` should remove the `davos` parser from 
    `input_transformers_post`
    """
    smuggle_func = davos.core.core.smuggle
    davos_parser = davos.implementations.full_parser
    # a reference
    transformers_list = IPYTHON_SHELL.input_transformers_post
    # a copy
    old_transformers_list = IPYTHON_SHELL.input_transformers_post[:]
    
    assert davos_parser in transformers_list
        
    try:
        davos.implementations.ipython_post7._deactivate_helper(smuggle_func, 
                                                               davos_parser)

        assert davos_parser not in transformers_list, transformers_list
    finally:
        # regardless of test outcome, make sure original 
        # input_transformers_post list and value of "smuggle" are 
        # restored
        IPYTHON_SHELL.input_transformers_post = old_transformers_list
        IPYTHON_SHELL.user_ns['smuggle'] = smuggle_func

In [ ]:
def test_deactivate_helper_deletes_smuggle():
    """
    `_deactivate_helper` should remove `"smuggle"` from the IPython user 
    namespace if it refers to `davos.core.core.smuggle`
    """
    smuggle_func = davos.core.core.smuggle
    davos_parser = davos.implementations.full_parser
    old_transformers_list = IPYTHON_SHELL.input_transformers_post[:]
    
    assert smuggle is IPYTHON_SHELL.user_ns['smuggle'] is smuggle_func
    
    try:
        davos.implementations.ipython_post7._deactivate_helper(smuggle_func, 
                                                               davos_parser)
        assert 'smuggle' not in IPYTHON_SHELL.user_ns
        with raises(NameError):
            smuggle os

    finally:
        IPYTHON_SHELL.user_ns['smuggle'] = smuggle_func
        IPYTHON_SHELL.input_transformers_post = old_transformers_list

In [ ]:
def test_deactivate_helper_leaves_smuggle():
    """
    running `_deactivate_helper` should *not* delete `"smuggle"` from 
    the IPython user namespace if it refers to something other than 
    `davos.core.core.smuggle`
    """
    global smuggle
    
    smuggle_func = davos.core.core.smuggle
    davos_parser = davos.implementations.full_parser
    old_transformers_list = IPYTHON_SHELL.input_transformers_post[:]
    
    smuggle = 'tmp-smuggle-val'
    print(smuggle_func)
    print(davos.config.ipython_shell.user_ns.get('smuggle'))
    try:
        davos.implementations.ipython_post7._deactivate_helper(smuggle_func, 
                                                               davos_parser)
        assert 'smuggle' in IPYTHON_SHELL.user_ns
        assert smuggle == 'tmp-smuggle-val'
    finally:
        IPYTHON_SHELL.user_ns['smuggle'] = smuggle_func
        IPYTHON_SHELL.input_transformers_post = old_transformers_list

In [ ]:
def test_deactivate_helper_not_active():
    """
    running `_deactivate_helper` shouldn't cause any problems if run 
    when the davos parser is already inactive
    """
    smuggle_func = davos.core.core.smuggle
    davos_parser = davos.implementations.full_parser
    
    davos.deactivate()

    try:
        for _ in range(5):
            davos.implementations.ipython_post7._deactivate_helper(smuggle_func, 
                                                                   davos_parser)
    finally:
        davos.activate()

In [ ]:
def test_generate_parser_func_side_effects():
    """
    The IPython >= 7.0 implementation of `generate_parser_func` should 
    set an attribute "`has_side_effects`" on the returned parser 
    function to `True`
    """
    davos_parser = davos.implementations.full_parser
    assert hasattr(davos_parser, 'has_side_effects')
    assert davos_parser.has_side_effects is True

In [ ]:
run_tests()